In [3]:
# import preprocess as prep
import model
import numpy as np
from datetime import datetime
import os
import importlib

# preprocess / load preprocessed data (preprocess.py)
# mini batching

# load model(can quickly change model by changing imported model)
# train / log
# generate checkpoints

# this will output 15 results/sec, how are we gonna handle it?

In [43]:
importlib.reload(model)

<module 'model' from 'c:\\Users\\jerry\\Desktop\\hly\\2024-2\\cap\\BN_SLR\\BomNae-SLR\\model.py'>

In [80]:
# utility functions

def bin2num(bin_arr):
    # big endian
    # lower index > higher exponent
    num = 0
    for val in bin_arr:
        num = (num << 1) | val
    return num

def num2bin(num, arr_len = -1):
    # big endian
    # lower index > higher exponent
    bin_arr = [int(x) for x in bin(num)[2:]]
    diff = arr_len - len(bin_arr)
    if diff < 1:
        return bin_arr
    else:
        pad = [0] * diff
        return pad+bin_arr

In [161]:
def num_arr2bin(num_arr, out_len = -1):
    # big endian
    # lower index > higher exponent
    res = []
    for num in num_arr:
        str_list = list(np.binary_repr(num).zfill(out_len))
        bin_arr = np.array(str_list, dtype=np.float32)
        res.append(bin_arr)
    return np.array(res)

In [129]:
# Configurations
load_size = 4 # number of data to be loaded at once
epochs = 8
save_dir = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints"
load_dir = "C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints"


In [1]:
# x_raw, y_raw = perp.load_data(something)
# x_raw is load_size number of full videos (array of frames)
# y_raw is a label
# y would be word number (index) for each x input
# example proof of concept
y_raw = np.array([2577, 2577, 2577,  312,312, 3, 3, 3, 3, 3])
y_train = num_arr2bin(y_raw, 12)
y_train

NameError: name 'np' is not defined

In [153]:
# dummy x_raw data
x_raw = []
for i in range(load_size):
    h_elem = np.zeros(shape=(4,5,3))
    # p_elem = np.zeros(shape=(3,10))
    # comb_elem = [h_elem, h_elem, p_elem]
    x_elem = []
    for j in range(np.random.randint(70,71)): # frame size
        x_elem.append(h_elem)
        # x_elem.append(comb_elem)
    x_raw.append(x_elem)

# dummy y_raw data
y_raw = np.random.randint(0,1024,load_size)

In [154]:
def serialize(vids, stride = 1):
    """input shape: (load_size, frames)
    ouput shape: (load_size, input_seq_size, 63 or 32, frames)"""
    each_size = []
    x_res = []
    for vid in vids:
        window_len = 0
        start = 0
        while (start + 63) < len(vid):
            x_res.append(vid[start: start+63: stride])
            window_len += 1
            start += 6
        each_size.append(window_len)
    return x_res, each_size

In [166]:
x_train, each = serialize(x_raw)
y_mid = np.repeat(y_raw, each)


In [139]:
np.array(x_train).shape

(4, 2, 63, 4, 5, 3)

In [156]:
np.array(x_train).shape

(8, 63, 4, 5, 3)

In [157]:
each

[2, 2, 2, 2]

In [159]:
y_mid

array([ 144,  144, 1011, 1011,  257,  257,  596,  596])

In [167]:
num_arr2bin(y_mid, 12)

array([[0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0.]], dtype=float32)

In [200]:
y_train.shape

(10, 12)

In [4]:
# dummy data for model testing
# hand input shape: (batch, time, h, w, channels)
# pose input shape: (batch, time, channel, features)
hand_l_dummy = np.zeros(shape=(256, 63, 4, 5, 3))
hand_r_dummy = np.zeros(shape=(256, 63, 4, 5, 3))
pose_dummy = np.zeros(shape=(256, 32, 3, 10))
x_dummy = (hand_l_dummy, hand_r_dummy, pose_dummy)

In [5]:
x_dummy[2].shape

(256, 32, 3, 10)

In [6]:
y_dummy1 = np.ones(shape=(256, 12))

In [7]:
y_dummy0 = np.zeros(shape=(256, 12))

In [40]:
model.reinit_model()

In [32]:
model.train(x_dummy, y_dummy1, 8)

Epoch 1/8
16/16 [==============================] - 0s 11ms/step - loss: 0.0238
Epoch 2/8
16/16 [==============================] - 0s 10ms/step - loss: 0.0147
Epoch 3/8
16/16 [==============================] - 0s 12ms/step - loss: 0.0101
Epoch 4/8
16/16 [==============================] - 0s 13ms/step - loss: 0.0074
Epoch 5/8
16/16 [==============================] - 0s 15ms/step - loss: 0.0057
Epoch 6/8
16/16 [==============================] - 0s 9ms/step - loss: 0.0046
Epoch 7/8
16/16 [==============================] - 0s 11ms/step - loss: 0.0038
Epoch 8/8
16/16 [==============================] - 0s 12ms/step - loss: 0.0031


In [47]:
pred = model.predict(x_dummy)

8/8 [==============================] - 0s 6ms/step


In [48]:
pred[0]

array([0.99877626, 0.9979128 , 0.9985296 , 0.9986358 , 0.9960804 ,
       0.995518  , 0.9969029 , 0.9925207 , 0.99776715, 0.9995015 ,
       0.996898  , 0.9965179 ], dtype=float32)

In [13]:
save_name = "asdf_test.keras"
save_path = os.path.join(save_dir, save_name)

In [35]:
model.save_model(save_path)

In [46]:
model.load_model(save_path)

Restored from C:/Users/jerry/Desktop/hly/2024-2/cap/BN_SLR/BomNae-SLR/checkpoints\asdf_test.keras\ckpt-1
